In [1]:
!pip install selenium
!pip install webdriver-manager


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
#Importing the necessary libraries

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException


In [18]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install()))

In [19]:
url = 'https://www.daraz.pk/'

In [20]:
driver.get(url)

In [21]:
search_bar = driver.find_element(By.XPATH, '/html/body/div[1]/div/div/div[1]/div/div/div[2]/div/div[2]/form/div/div[1]/input[1]')
search_bar.send_keys('Redmi Note 12' + Keys.RETURN)

In [25]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
#driver.back()
ad_grid = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')
ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')
# Loop through each ad card, click on it, and then return to the previous page
price_mobile = []
dilivery_status = []
ratings_list = []
seller_ratings_list = []
Shipping_performance = []
flagship_mall = []
scores = []
product_name = []
for index in range(0,2):
    # Find the ad cards again to avoid StaleElementReferenceException
    ad_grid = driver.find_element(By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')
    ad_cards = ad_grid.find_elements(By.CLASS_NAME, 'gridItem--Yd0sa')
    
    # Click on the current ad card
    current_card = ad_cards[index]
    current_card.click()
    Information_grid = driver.find_element(By.XPATH, '/html/body/div[4]/div/div[3]')
    prices = Information_grid.find_elements(By.CLASS_NAME, 'pdp-product-price')
    price_ = Information_grid.find_element(By.XPATH, '//*[@id="module_product_price_1"]/div/div/span')
    price_mobile.append(price_.text)
    

    dilivery_bar = driver.find_element(By.XPATH, '//*[@id="block-Cve5yP08rW9"]')
    dilivery_opt = driver.find_elements(By.CLASS_NAME, 'delivery-option-item__title')
    Seller_rating = dilivery_bar.find_element(By.XPATH, '/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[1]/div[2]')
    Shipping_perf = dilivery_bar.find_element(By.XPATH, '//*[@id="module_seller_info"]/div/div[2]/div[2]/div[2]')
    flagship_mall_xpath = '//*[@id="module_seller_info"]/div/div[1]/div[1]/div[2]/a[2]/img'
    name_of_product = Information_grid.find_element(By.XPATH, '//*[@id="module_product_title_1"]/div/div')
    product_name.append(name_of_product.text)
    flagship_mall_element = dilivery_bar.find_elements(By.XPATH, flagship_mall_xpath)
    if flagship_mall_element:
        flagship_mall.append(1)
    else:
        flagship_mall.append(0)
    seller_ratings_list.append(Seller_rating.text)
    Shipping_performance.append(Shipping_perf.text)
    driver.execute_script("window.scrollTo(0, 500);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(500, 800);")
    time.sleep(1)
    driver.execute_script("window.scrollTo(800, 1000);")
    time.sleep(2)
    rating_bar = driver.find_element(By.XPATH, '//*[@id="module_product_review"]')
    rating = rating_bar.find_element(By.XPATH, '//*[@id="module_product_review"]/div/div/div[1]/div[2]/div/div/div[1]/div[1]/span[1]')
    ratings_list.append(rating.text)
    counter = 0
    for i in dilivery_opt:
        if 'Free Delivery' in (i.text):
            counter+=1
            dilivery_status.append(1)
    if(counter==0):
        dilivery_status.append(0)
    
    # Calculate the base score (price)
    base_score = int(price_mobile[index].replace('Rs. ', '').replace(',', ''))

    # Check if the product has "Daraz Mall" status
    daraz_mall_score = 5000 if flagship_mall[index] == 1 else 0

    # Check if the product has free shipping
    free_shipping_score = 1000 if dilivery_status[index] == 1 else 0
    product_rating = float(ratings_list[index])
    seller_rating = float(seller_ratings_list[index].replace('%', ''))
    shipping_performance = float(Shipping_performance[index].replace('%', ''))
    score = base_score + daraz_mall_score + free_shipping_score + 10000 * (
        (0.4 * (product_rating / 5)) + (0.4 * (seller_rating / 100)) + (0.2 * (shipping_performance / 100))
    )
    
    scores.append(score)

    time.sleep(2)  # Add a short delay to allow the page to load


    # Go back to the previous page
    driver.back()
    # Wait for the ad grid to be present again to avoid StaleElementReferenceException
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')))
# print(price_mobile)
# print(dilivery_status)
# print(ratings_list)
# print(seller_ratings_list)
# print(Shipping_performance)
# print(flagship_mall)
best_product_index = scores.index(max(scores))

# Print the details of the best product
print(f"Best Product Details:")
print(f"Name: {product_name[best_product_index]}")
print(f"Price: {price_mobile[best_product_index]}")
print(f"Daraz Mall Status: {'Yes' if flagship_mall[best_product_index] == 1 else 'No'}")
print(f"Free Shipping: {'Yes' if dilivery_status[best_product_index] == 1 else 'No'}")
print(f"Product Rating: {ratings_list[best_product_index]}")
print(f"Seller Rating: {seller_ratings_list[best_product_index]}")
print(f"Shipping Performance: {Shipping_performance[best_product_index]}")
print(f"Total Score: {scores[best_product_index]}")

Best Product Details:
Name: Xiaomi Redmi Note 12 || 8GB Ram 128GB Rom || 5000mAh Battery 33W Fast Charger
Price: Rs. 51,499
Daraz Mall Status: No
Free Shipping: Yes
Product Rating: 4.7
Seller Rating: 91%
Shipping Performance: 82%
Total Score: 61539.0


In [47]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

def find_element(xpath, parent=None):
    if parent:
        return parent.find_element(By.XPATH, xpath)
    return driver.find_element(By.XPATH, xpath)

def find_elements(xpath, parent=None):
    if parent:
        return parent.find_elements(By.XPATH, xpath)
    return driver.find_elements(By.XPATH, xpath)

def scroll_down(height):
    driver.execute_script(f"window.scrollTo(0, {height});")
    time.sleep(1)

def scrape_product(index):
    current_card = ad_cards[index]
    current_card.click()
    information_grid = find_element('/html/body/div[4]/div/div[3]')
    price_mobile.append(find_element('//*[@id="module_product_price_1"]/div/div/span', information_grid).text)
    dilivery_bar = find_element('//*[@id="block-Cve5yP08rW9"]', information_grid)
    dilivery_opt = find_elements('.//*[contains(@class, "delivery-option-item__title")]', dilivery_bar)
    seller_rating = find_element('/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[1]/div[2]', dilivery_bar).text
    Shipping_perf = find_element('//*[@id="module_seller_info"]/div/div[2]/div[2]/div[2]', dilivery_bar).text
    flagship_mall_element = find_elements('//*[@id="module_seller_info"]/div/div[1]/div[1]/div[2]/a[2]/img', dilivery_bar)
    name_of_product = find_element('//*[@id="module_product_title_1"]/div/div', information_grid).text
    product_name.append(name_of_product)
    flagship_mall.append(1) if flagship_mall_element else flagship_mall.append(0)
    seller_ratings_list.append(seller_rating)
    Shipping_performance.append(Shipping_perf)
    scroll_down(500)
    scroll_down(800)
    scroll_down(1000)
    rating_bar = find_element('//*[@id="module_product_review"]', information_grid)
    rating = find_element('//*[@id="module_product_review"]/div/div/div[1]/div[2]/div/div/div[1]/div[1]/span[1]', rating_bar).text
    ratings_list.append(rating)
    counter = sum('Free Delivery' in i.text for i in dilivery_opt)
    dilivery_status.append(1) if counter > 0 else dilivery_status.append(0)
    scores.append(calculate_score(index))
    time.sleep(2)
    driver.back()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')))

def calculate_score(product_index):
    try:
        base_score = int(price_mobile[product_index].replace('Rs. ', '').replace(',', ''))
    except (ValueError, IndexError):
        base_score = 0

    daraz_mall_score = 5000 if flagship_mall[product_index] == 1 else 0
    free_shipping_score = 1000 if dilivery_status[product_index] == 1 else 0

    try:
        product_rating = float(ratings_list[product_index])
    except (ValueError, IndexError):
        product_rating = 0.0

    try:
        seller_rating = float(seller_ratings_list[product_index].replace('%', ''))
    except (ValueError, IndexError):
        seller_rating = 0.0

    try:
        shipping_performance = float(Shipping_performance[product_index].replace('%', ''))
    except (ValueError, IndexError):
        shipping_performance = 0.0

    return base_score + daraz_mall_score + free_shipping_score + 10000 * (
        (0.4 * (product_rating / 5)) + (0.4 * (seller_rating / 100)) + (0.2 * (shipping_performance / 100))
    )

# Your existing code...
# Your existing code...

# Set the range of elements you want to process
start_index = 4 # You can change this to the desired starting index
end_index = 7  # You can change this to the desired ending index

for index in range(start_index, end_index):
    ad_grid = find_element('//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')
    ad_cards = find_elements('.//*[contains(@class, "gridItem--Yd0sa")]', ad_grid)
    
    # Check if the index is within the valid range
    if index < len(ad_cards):
        scrape_product(index)
    else:
        print(f"Skipping index {index} as it is out of range.")


best_product_index = scores.index(max(scores))
# Print the details of the best product
print(f"Best Product Details:")
print(f"Name: {product_name[best_product_index]}")
print(f"Price: {price_mobile[best_product_index]}")
print(f"Daraz Mall Status: {'Yes' if flagship_mall[best_product_index] == 1 else 'No'}")
print(f"Free Shipping: {'Yes' if dilivery_status[best_product_index] == 1 else 'No'}")
print(f"Product Rating: {ratings_list[best_product_index]}")
print(f"Seller Rating: {seller_ratings_list[best_product_index]}")
print(f"Shipping Performance: {Shipping_performance[best_product_index]}")
print(f"Total Score: {scores[best_product_index]}")



Best Product Details:
Name: Redmi Note 12 8GB Ram 128GB Rom, Display Size 6.67 Inches, Battery Capacity 5000 mAh , Fast charging 33W
Price: Rs. 53,999
Daraz Mall Status: No
Free Shipping: Yes
Product Rating: 4.6
Seller Rating: 94%
Shipping Performance: 80%
Total Score: 99560.0


In [ ]:
import time
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
price_mobile = []
dilivery_status = []
ratings_list = []
seller_ratings_list = []
Shipping_performance = []
flagship_mall = []
scores = []
product_name = []
def find_element(xpath, parent=None):
    if parent:
        return parent.find_element(By.XPATH, xpath)
    return driver.find_element(By.XPATH, xpath)

def find_elements(xpath, parent=None):
    if parent:
        return parent.find_elements(By.XPATH, xpath)
    return driver.find_elements(By.XPATH, xpath)

def scroll_down(height):
    driver.execute_script(f"window.scrollTo(0, {height});")
    time.sleep(1)

def scrape_product(index):
    current_card = ad_cards[index]
    current_card.click()
    information_grid = find_element('/html/body/div[4]/div/div[3]')
    price_mobile.append(find_element('//*[@id="module_product_price_1"]/div/div/span', information_grid).text)
    dilivery_bar = find_element('//*[@id="block-Cve5yP08rW9"]', information_grid)
    dilivery_opt = find_elements('.//*[contains(@class, "delivery-option-item__title")]', dilivery_bar)
    seller_rating = find_element('/html/body/div[4]/div/div[3]/div[2]/div/div[2]/div[5]/div/div[2]/div[1]/div[2]', dilivery_bar).text
    Shipping_perf = find_element('//*[@id="module_seller_info"]/div/div[2]/div[2]/div[2]', dilivery_bar).text
    flagship_mall_element = find_elements('//*[@id="module_seller_info"]/div/div[1]/div[1]/div[2]/a[2]/img', dilivery_bar)
    name_of_product = find_element('//*[@id="module_product_title_1"]/div/div', information_grid).text
    product_name.append(name_of_product)
    flagship_mall.append(1) if flagship_mall_element else flagship_mall.append(0)
    seller_ratings_list.append(seller_rating)
    Shipping_performance.append(Shipping_perf)
    scroll_down(500)
    scroll_down(800)
    scroll_down(1000)
    scroll_down(1200)
    rating_bar = find_element('//*[@id="module_product_review"]', information_grid)
    rating = rating_bar.find_element(By.XPATH, '/html/body/div[4]/div/div[9]/div[1]/div[2]/div/div/div/div[1]/div[2]/div/div/div[1]/div[1]/span[1]').text
    ratings_list.append(rating)
    counter = sum('Free Delivery' in i.text for i in dilivery_opt)
    dilivery_status.append(1) if counter > 0 else dilivery_status.append(0)
    print(index)
    scores.append(calculate_score(index))
    time.sleep(2)
    driver.back()
    WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')))

def calculate_score(product_index):
    try:
        base_score = int(price_mobile[product_index].replace('Rs. ', '').replace(',', ''))
    except (ValueError, IndexError):
        base_score = 0

    daraz_mall_score = 5000 if flagship_mall[product_index] == 1 else 0
    free_shipping_score = 1000 if dilivery_status[product_index] == 1 else 0

    try:
        product_rating = float(ratings_list[product_index])
    except (ValueError, IndexError):
        product_rating = 0.0

    try:
        seller_rating = float(seller_ratings_list[product_index].replace('%', ''))
    except (ValueError, IndexError):
        seller_rating = 0.0

    try:
        shipping_performance = float(Shipping_performance[product_index].replace('%', ''))
    except (ValueError, IndexError):
        shipping_performance = 0.0

    return base_score + daraz_mall_score + free_shipping_score + 10000 * (
        (0.4 * (product_rating / 5)) + (0.4 * (seller_rating / 100)) + (0.2 * (shipping_performance / 100))
    )

# Set the range of elements you want to process
start_index = 0 # You can change this to the desired starting index
end_index = 3  # You can change this to the desired ending index

driver.back()
for index in range(start_index, end_index):
    driver.execute_script(f"window.scrollTo(0, 100);")
    ad_grid = find_element('//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')

    # Scroll down and wait for new elements to load
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)  # Adjust the sleep duration based on the page loading time

    ad_cards = find_elements('.//*[contains(@class, "gridItem--Yd0sa")]', ad_grid)

    # Check if the index is within the valid range
    if index < len(ad_cards):
        scrape_product(index)
        # Reinitialize ad_cards after going back
        ad_grid = find_element('//*[@id="root"]/div/div[3]/div/div/div[1]/div[2]')
        ad_cards = find_elements('.//*[contains(@class, "gridItem--Yd0sa")]', ad_grid)
    else:
        print(f"Skipping index {index} as it is out of range.")

best_product_index = scores.index(max(scores))

# Print the details of the best product
print(f"Best Product Details:")
print(f"Name: {product_name[best_product_index]}")
print(f"Price: {price_mobile[best_product_index]}")
print(f"Daraz Mall Status: {'Yes' if flagship_mall[best_product_index] == 1 else 'No'}")
print(f"Free Shipping: {'Yes' if dilivery_status[best_product_index] == 1 else 'No'}")
print(f"Product Rating: {ratings_list[best_product_index]}")
print(f"Seller Rating: {seller_ratings_list[best_product_index]}")
print(f"Shipping Performance: {Shipping_performance[best_product_index]}")
print(f"Total Score: {scores[best_product_index]}")